# Google Search Console GSC API Examples


### Overview

If you are interested in your organic traffic then the Google Search Console is hands down 
one of the richest datasources.

Here are example scripts for connecting to this valuable datasource.


> SEE PART 1 for how to connect and download GSC data



### About Me

My name is Alton Alexander. I am a Data Science consultant turned entreprenuer building SaaS tools for SEO.

Find more about my free scripts or ask me any question on twitter: @alton_lex

# GSC API Examples:

In [40]:
# load libraries
import requests
import json
from urllib.parse import urlparse

import httplib2
from apiclient import errors
from apiclient.discovery import build

import datetime

from google.oauth2 import service_account
import google.oauth2.credentials
import google.auth.transport.requests

!pip install pandas
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


## 0) Setup service account

In Google Cloud Console you need to create the following resources:

1. A project

2. Enable Access to the GSC API

3. A service account (make note of the Email for the service account)

4. OPTIONAL Add rolls to the service account so that it can access additional resources (ie. BigQuery)

5. Save the Key as JSON for connecting from python


In Google Search Console:

1. add the service account email as a user to each of the properties you want it to access

## 1) Connect with Service Account JSON key

Every request to the GSC API must have user authentication.

We use the service account JSON Key we get credentials.


In [104]:
# local path to the service account key
# service account's email must be added to one or more properties in GSC

SERVICE_ACCOUNT_FILE = "../gcp-keys/website-analytics-161019-16456165cddc.json"
#SERVICE_ACCOUNT_FILE = "./service-account-key.json"

In [105]:
SCOPES = ['https://www.googleapis.com/auth/webmasters']
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [106]:
service = build(
    'webmasters',
    'v3',
    credentials=credentials
)

In [107]:
# test by pulling a list of sites
# reminder to add the service account email to every GSC property as a user


response = service.sites().list().execute()
response

{'siteEntry': [{'siteUrl': 'sc-domain:frontanalytics.com',
   'permissionLevel': 'siteFullUser'}]}

## 1) Pull a list of pages

This pulls a list of all the pages that are showing on Google Search in the last 30 days:

In [45]:
# define the domain that we are using

# hardcode the domain for the property
#site = "frontanalytics.com"

# or select the first property from the response above
site = response['siteEntry'][0]['siteUrl'].split(":")[1]
site

'frontanalytics.com'

In [46]:
# set date for the last 30 days
today = datetime.datetime.today()
startDate = today - datetime.timedelta(days=32)
endDate = today - datetime.timedelta(days=2)

In [47]:
# by page

# recent
# Set the dates in this format "2022-12-01"
data = {
  "startDate": startDate.strftime("%Y-%m-%d"),
  "endDate": endDate.strftime("%Y-%m-%d"),
  "dimensions": "page"
}
res = requests.post("https://www.googleapis.com/webmasters/v3/sites/"+"sc-domain:"+site+"/searchAnalytics/query?access_token="+credentials.token, json=data)

In [48]:
j = json.loads(res.text)['rows']
for i in range(len(j)):
    j[i]['url'] = j[i]['keys'][0]

df_pages = pd.DataFrame(j)

df_pages = df_pages.sort_values('impressions',ascending=False)
df_pages

,keys,clicks,impressions,ctr,position,url
0,[https://frontanalytics.com/],9,220,0.040909,9.150000,https://frontanalytics.com/
4,[https://frontanalytics.com/advanced-analytics/],0,107,0.000000,33.607477,https://frontanalytics.com/advanced-analytics/
1,[https://frontanalytics.com/system-dynamics-mo...,9,104,0.086538,49.701923,https://frontanalytics.com/system-dynamics-mod...
8,[https://frontanalytics.com/inbound-call-cente...,0,74,0.000000,53.689189,https://frontanalytics.com/inbound-call-center...
2,[https://frontanalytics.com/about/],0,53,0.000000,4.679245,https://frontanalytics.com/about/
5,[https://frontanalytics.com/contact/],0,52,0.000000,3.096154,https://frontanalytics.com/contact/
9,[https://frontanalytics.com/privacy/],0,52,0.000000,3.076923,https://frontanalytics.com/privacy/
6,[https://frontanalytics.com/data-monetization/],0,5,0.000000,21.800000,https://frontanalytics.com/data-monetization/
7,[https://frontanalytics.com/digital-strategy/],0,3,0.000000,36.333333,https://frontanalytics.com/digital-strategy/
10,[https://frontanalytics.com/system-dynamics-fo...,0,2,0.000000,67.000000,https://frontanalytics.com/system-dynamics-for...


## 2) Get all KWs for each page

This pulls every query used for each page:

In [120]:
# set date for the previous 1 days (day before yesterday)
updated_at = datetime.datetime.today()
today = updated_at.date()
startDate = today - datetime.timedelta(days=4)
endDate = today - datetime.timedelta(days=3)

df_all_queries = pd.DataFrame()

# Get all the queries
for index, row in df_pages.iterrows():
    
    # get the url of this page
    page_url = row['url']

    # recent
    data = {
      "startDate": startDate.strftime("%Y-%m-%d"),
      "endDate": endDate.strftime("%Y-%m-%d"),
      "dimensions": ["query","device","country"],
      "dimensionFilterGroups": [
        {
          "groupType": "and",
          "filters": [
            {
              "dimension": "page",
              "operator": "contains",
              "expression": page_url
            }
          ]
        }
      ]
    }
    res = requests.post("https://www.googleapis.com/webmasters/v3/sites/"+"sc-domain:"+site+"/searchAnalytics/query?access_token="+credentials.token, json=data)

    # convert the response to a data frame
    j = json.loads(res.text).get('rows',[])

    if(len(j)):
        df_queries = pd.DataFrame(j)
        df_queries['url'] = page_url
        df_queries['property'] = site
        df_queries['start_date'] = startDate
        df_queries['updated_at'] = updated_at

        # By default the keys/dimensions are in a single column, let's split them out into separate columns.
        new_cols = df_queries['keys'].astype(str).str.replace("[","").str.replace("]","")
        new_cols = new_cols.str.split(pat=',',expand=True,n=2)

        # Give the columsn sensible names
        new_cols.columns = ["query","device","country"]

        # Bring back a key from the intial dataframe so we can join
        new_cols['key'] = df_queries['keys']

        # Get rid of quotation marks
        new_cols['query'] = new_cols['query'].str.replace("'","").str.lower()
        new_cols['device'] = new_cols['device'].str.replace("'","").str.lower()
        new_cols['country'] = new_cols['country'].str.replace("'","").str.lower()

        # Join in the new clean columns to our intiial data
        df_queries = pd.concat([df_queries, new_cols], axis=1, join='inner')
        
        # Drop the key column and ctr
        df_queries = df_queries.drop(["key","keys", "ctr"],axis=1)

        # save all the queries for this page with all other pages
        df_all_queries = pd.concat([df_all_queries, df_queries])

/tmp/ipykernel_3601931/3410792742.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_cols = df_queries['keys'].astype(str).str.replace("[","").str.replace("]","")
/tmp/ipykernel_3601931/3410792742.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_cols = df_queries['keys'].astype(str).str.replace("[","").str.replace("]","")
/tmp/ipykernel_3601931/3410792742.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_cols = df_queries['keys'].astype(str).str.replace("[","").str.replace("]","")
/tmp/ipykernel_3601

In [121]:
# Now you can save df_all_queries for additional analysis
df_all_queries

,clicks,impressions,position,url,property,start_date,updated_at,query,device,country
0,1,1,3,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,front analytics,desktop,gbr
1,0,1,21,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,advanced analytics generally refers to,desktop,chl
2,0,1,21,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,advanced analytics generally refers to,desktop,col
3,0,1,21,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,advanced analytics generally refers to,desktop,rus
4,0,2,23,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,advanced analytics generally refers to,desktop,usa
5,0,1,41,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,advanced analytics refers to,desktop,usa
6,0,1,95,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,call center analytics case study,mobile,irl
7,0,1,29,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,call center optimization case study,desktop,mys
8,0,1,29,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,call center optimization case study,desktop,usa
9,0,1,88,https://frontanalytics.com/,frontanalytics.com,2023-01-10,2023-01-14 10:17:21.991669,define system dynamics,desktop,jpn


# Configure BigQuery

In [122]:
!pip install --upgrade google-cloud
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-storage
from google.cloud import bigquery

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [123]:

# establish a BigQuery client
client_bq = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)

In [124]:
# connect to big query and list all datasets

datasets = client_bq.list_datasets()

for dataset in datasets:
    print(dataset.dataset_id)

data_winners_dataset
snowplow_analytics
test


In [125]:
# get the project name from the service account

BQ_PROJECT_NAME = client_bq.project
BQ_PROJECT_NAME

'website-analytics-161019'

## Create the BQ Dataset and Table

In [128]:
# create the dataset and table if they don't exist yet

BQ_DATASET_NAME = 'data_winners_dataset'
BQ_TABLE_NAME = 'gsc_daily_table'


# create the dataset
try:
    # Construct a full Dataset object to send to the API.
    dataset = bigquery.Dataset("{}.{}".format(client_bq.project, BQ_DATASET_NAME))

    # TODO(developer): Specify the geographic location where the dataset should reside.
    dataset.location = "US"

    # Send the dataset to the API for creation, with an explicit timeout.
    # Raises google.api_core.exceptions.Conflict if the Dataset already
    # exists within the project.
    dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client_bq.project, dataset.dataset_id))
except Exception as e:
    print(e)

# create the table
# https://cloud.google.com/bigquery/docs/samples/bigquery-create-table 
try:
    # TODO(dev): Change table_id to the full name of the table you want to create.
    #table_id = "your-project.your_dataset.your_table_name"
    table_id = "{}.{}.{}".format(client_bq.project, BQ_DATASET_NAME, BQ_TABLE_NAME)
    
    schema = [
        bigquery.SchemaField("property", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("start_date", "DATE", mode="REQUIRED"),
        bigquery.SchemaField("url", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("query", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("device", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("country", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("clicks", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("impressions", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("position", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("updated_at", "DATETIME", mode="REQUIRED"),
    ]

    table_description = "Stores GSC data every day. Auto generated via data-winners. For details see https://github.com/FrontAnalyticsInc/data-winners/tree/main/datasource-api-google-search-console"
    
    table = bigquery.Table(table_id, schema=schema)
    table.description = table_description
    table = client.create_table(table)  # API request

    print(f"Created {table_id}.")
except Exception as e:
    print(e)

Created dataset website-analytics-161019.data_winners_dataset
Created website-analytics-161019.data_winners_dataset.gsc_daily_table.


# Test Uploading to BQ

Make sure your service account has a role that permits access to BigQuery

In [130]:

# Set the destination table
table_id = '{}.{}.{}'.format(BQ_PROJECT_NAME, BQ_DATASET_NAME, BQ_TABLE_NAME)


# create a load job config
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        #bigquery.SchemaField("start_date", bigquery.enums.SqlTypeNames.DATE),
        #bigquery.SchemaField("updated_at", bigquery.enums.SqlTypeNames.DATETIME),
        # Indexes are written if included in the schema by name.
        #bigquery.SchemaField("wikidata_id", bigquery.enums.SqlTypeNames.STRING),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_APPEND",
)

job = client_bq.load_table_from_dataframe(
    df_all_queries, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.



LoadJob<project=website-analytics-161019, location=US, id=03026fe2-30ad-4cc1-badd-5167e83b5dd7>

# Preparing for Automation

In [131]:
!pip install pandas-gbq
!pip install db-dtypes

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [133]:
# what has already been loaded?
import dateutil

def query_table(start_date, PROJECT_ID, DATASET, TABLE):
    QUERY = "SELECT * FROM {dataset}.{table} WHERE start_date = '{d}'"
    
    query = QUERY.format(dataset=DATASET, table=TABLE, d=start_date)
    result = pd.read_gbq(query, PROJECT_ID, dialect='standard')

    result['already_loaded'] = True
    return result


# get all the records already loaded within the last n days
already_loaded = query_table("2023-01-10", BQ_PROJECT_NAME, BQ_DATASET_NAME, BQ_TABLE_NAME)

already_loaded

,property,start_date,url,query,device,country,clicks,impressions,position,updated_at,already_loaded
0,frontanalytics.com,2023-01-10,https://frontanalytics.com/,front analytics,desktop,gbr,1,1,3,2023-01-14 10:17:21.991669,True
1,frontanalytics.com,2023-01-10,https://frontanalytics.com/,front analytics,desktop,deu,0,1,3,2023-01-14 10:17:21.991669,True
2,frontanalytics.com,2023-01-10,https://frontanalytics.com/,front analytics,desktop,usa,0,8,3,2023-01-14 10:17:21.991669,True
3,frontanalytics.com,2023-01-10,https://frontanalytics.com/about/,front analytics,desktop,usa,0,2,3,2023-01-14 10:17:21.991669,True
4,frontanalytics.com,2023-01-10,https://frontanalytics.com/contact/,front analytics,desktop,usa,0,2,3,2023-01-14 10:17:21.991669,True
5,frontanalytics.com,2023-01-10,https://frontanalytics.com/privacy/,front analytics,desktop,usa,0,2,3,2023-01-14 10:17:21.991669,True
6,frontanalytics.com,2023-01-10,https://frontanalytics.com/,in front analytics,desktop,fra,0,1,5,2023-01-14 10:17:21.991669,True
7,frontanalytics.com,2023-01-10,https://frontanalytics.com/,in front analytics,desktop,idn,0,1,5,2023-01-14 10:17:21.991669,True
8,frontanalytics.com,2023-01-10,https://frontanalytics.com/,in front analytics,mobile,ita,0,1,7,2023-01-14 10:17:21.991669,True
9,frontanalytics.com,2023-01-10,https://frontanalytics.com/,front data,desktop,fra,0,1,8,2023-01-14 10:17:21.991669,True


In [144]:
already_loaded[join_cols]

,start_date,property,url,device,country,already_loaded
0,2023-01-10,frontanalytics.com,https://frontanalytics.com/,desktop,gbr,True
1,2023-01-10,frontanalytics.com,https://frontanalytics.com/,desktop,deu,True
2,2023-01-10,frontanalytics.com,https://frontanalytics.com/,desktop,usa,True
3,2023-01-10,frontanalytics.com,https://frontanalytics.com/about/,desktop,usa,True
4,2023-01-10,frontanalytics.com,https://frontanalytics.com/contact/,desktop,usa,True
5,2023-01-10,frontanalytics.com,https://frontanalytics.com/privacy/,desktop,usa,True
6,2023-01-10,frontanalytics.com,https://frontanalytics.com/,desktop,fra,True
7,2023-01-10,frontanalytics.com,https://frontanalytics.com/,desktop,idn,True
8,2023-01-10,frontanalytics.com,https://frontanalytics.com/,mobile,ita,True
9,2023-01-10,frontanalytics.com,https://frontanalytics.com/,desktop,fra,True


In [145]:
# find the rows that aren't loaded yet
join_cols = ['start_date','property','url','device','country']
not_loaded = df_all_queries.merge(already_loaded[join_cols+['already_loaded']], how="left", on=join_cols)
not_loaded = not_loaded[not_loaded['already_loaded'].isna()]
cols = list(df_all_queries.columns)
not_loaded = not_loaded[cols]
not_loaded

,clicks,impressions,position,url,property,start_date,updated_at,query,device,country
